In [36]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['figure.facecolor'] = 'white'
matplotlib.rcParams['savefig.dpi'] = 300
from matplotlib import cm
from scipy.integrate import solve_ivp


import seaborn as sns

from tqdm.auto import tqdm

import os
import ctypes
import gc

In [37]:
solve_ivp

<function scipy.integrate._ivp.ivp.solve_ivp(fun, t_span, y0, method='RK45', t_eval=None, dense_output=False, events=None, vectorized=False, args=None, **options)>

In [38]:
#  https://stackoverflow.com/a/37664693/13213091
def wrapped_ndptr(*args, **kwargs):
    base = np.ctypeslib.ndpointer(*args, **kwargs)
    def from_param(cls, obj):
        if obj is None:
            return obj
        return base.from_param(obj)
    return type(base.__name__, (base,), {'from_param': classmethod(from_param)})
DoubleArrayType_1D = wrapped_ndptr(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS')
DoubleArrayType_2D = wrapped_ndptr(dtype=np.float64, ndim=2, flags='C_CONTIGUOUS')

In [39]:
dirname = '../src/model_ctypes/_bondarenko'
filename_so = os.path.join(dirname, 'model.so')

filename_so_abs = os.path.abspath(filename_so)

model = ctypes.CDLL(filename_so_abs)

model.run.argtypes = [
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'), # double *S
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'), # double *C
    ctypes.c_int, # int n_beats
    ctypes.c_double, # double t_sampling
    ctypes.c_double, # double tol
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=2, flags='C_CONTIGUOUS'), # double *output
    
    DoubleArrayType_2D, #np.ctypeslib.ndpointer(dtype=np.float64, ndim=2, flags='C_CONTIGUOUS'), # double *output_A
    DoubleArrayType_1D, #np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS') # double *output_t
    DoubleArrayType_1D, # double *stim_protocol
]

model.run.restype = ctypes.c_int

model.fun.argtypes = [
    ctypes.c_double,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS')
]

def run(S, C, n_beats, t_sampling, tol, output, *,
        output_A=None, t=None, stim_protocol=None):
    
    return model.run(S, C, n_beats, t_sampling, tol, output, output_A, t, stim_protocol)


legend_constants = pd.read_csv(os.path.join(dirname, "legend_constants.csv"), index_col='name')['value']
legend_states = pd.read_csv(os.path.join(dirname, "legend_states.csv"), index_col='name')['value']
# legend_algebraic = pd.read_csv(os.path.join(dirname, "legend_algebraic.csv"), index_col='name')
# legend_algebraic['value'] = 0.0
# legend_algebraic = legend_algebraic['value']

# model.initialize_states_default(legend_states.values, legend_constants.values)
# legend_states.to_csv(os.path.join(dirname, "legend_states.csv"))

In [40]:
S = legend_states.copy()
C = legend_constants.copy()
A = np.zeros(6)

In [41]:
A

array([0., 0., 0., 0., 0., 0.])

In [42]:
ydot = np.zeros_like(S)
y = S.values.copy()
params = C.values.copy()
t = 0.42

In [43]:
%%timeit
model.fun(t, y, ydot, A, params)

14.5 µs ± 47.1 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [49]:
def fun(t, y, params):
    ydot = np.zeros_like(y)
    model.fun(t, y, ydot, A, params)
    # ydot[[7, 16, 19, 118]] = 0  # these are stiff guys
    return ydot

In [45]:
CL = 250

C['CL'] = CL
n_beats = 1
stim_period = CL
t_sampling = 1.

t_space = np.linspace(0, stim_period * n_beats, int(stim_period / t_sampling) * n_beats + 1, endpoint=True)
t_span = 0, t_space[-1]

In [52]:
%%timeit
sol = solve_ivp(fun, y0=S,
                t_span=t_span,# t_eval=t_space,
                args=(C.values.copy(),),
                method = 'LSODA', # atol=atol, # rtol=1e-3,
                #max_step=1. * t_sampling,
                )

422 ms ± 9.89 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
